In [1]:
import numpy as np
import scipy
from scipy.special import logsumexp

In [2]:
X = np.load("X.npy")

In [7]:
a0 = np.array([0.5,0.4,0.1])
b0 = np.array([0.5,0.4,0.1])
c0 = np.array([0.5,0.4,0.1])
#pi0 = [float(1)/(d-3)] * (d-3)

In [18]:
a1 = np.array([0.9,0.05,0.05])
b1 = np.array([0.9,0.05,0.05])
c1 = np.array([0.9,0.05,0.05])

In [14]:
def my_E(X,a,b,c,pi):
    n,d = X.shape
    V = np.zeros([n,d-3])
    W = np.zeros([n,d-3])
    for k in range(d-3):
        #print(k)
        loga = np.log(a[X[:,k]]) + np.log(a[X[:,k+1]])
        logb = np.log(b[X[:,k+2]]) + np.log(b[X[:,k+3]])        
        logc = np.sum(np.array([np.log(c[X[:,p]]) for p in range(d) if p not in range(k,k+4)]), axis = 0)
        V[:,k] = np.log(pi[k]) + loga+logb+logc
    
    dd = logsumexp(V, axis = 1)
    #print(dd.shape)
    W = np.exp(V - dd[:,None])

    return(W)
    

In [26]:
def my_EM(X=X, eps = 0.001, a = a0, b = b0, c=c0):
    n,d = X.shape
    
    ## initializatio:
    #a = np.array([0.5,0.4,0.1])
    #b = np.array([0.5,0.4,0.1])
    #c = np.array([0.5,0.4,0.1]) 
    
    theta = np.concatenate((a,b,c))
    
    pi = [float(1)/(d-3)] * (d-3)
    
    N = 1000
    
    ## loop
    for step in range(N):
        #print(step)
        ## E step 
        W = my_E(X=X,a=a,b=b,c=c,pi=pi)
        
        pi = W.mean(axis = 0)
        
        
        ## store necessary info in am, bm,cm
        am = np.zeros([n,d-3,3])
        bm = am.copy()
        cm = am.copy()        
        for k in range(d-3):
            for i in range(3):
                am[:,k,i] = (X[:,k:k+2] == i).sum(axis = 1) * W[:,k]
                bm[:,k,i] = (X[:,k+2:k+4] == i).sum(axis = 1) * W[:,k]
                cm[:,k,i] = ((X[:,:k] == i).sum(axis = 1)+
                             (X[:,k+4:] == i).sum(axis = 1)) * W[:,k]
        
        ## use am, bm, cm to update a,b,c
        for i in range(3):
            a[i] = am[:,:,i].sum()/am.sum()
            #print(a[i])
            b[i] = bm[:,:,i].sum()/bm.sum()
            c[i] = cm[:,:,i].sum()/cm.sum()
            
        ## compare progress with  eps
        theta_old = theta.copy()
        theta = np.concatenate((a,b,c))
        
        prog = np.linalg.norm(theta-theta_old)/np.linalg.norm(theta_old)
        if prog < eps:
            print("break after step: "+ str(k))
            break
    
    
    return(a,b,c,pi)
            
        
            
    
    
    
    

In [27]:
af,bf,cf,pi = my_EM()

break after step: 16


In [28]:
print("a:")
print(af)
print("b:")
print(bf)
print("c:")
print(cf)
print("pi:")
print(pi)

a:
[0.02949659 0.00550341 0.965     ]
b:
[1.50770244e-01 8.49229755e-01 6.02883288e-10]
c:
[9.07466646e-01 9.25333537e-02 6.71804088e-13]
pi:
[0.27       0.15       0.05932878 0.08067122 0.09       0.05
 0.06       0.01       0.03017789 0.04964423 0.02017789 0.03
 0.00966439 0.01033561 0.02017789 0.04982211 0.01      ]


In [29]:
af1,bf1,cf1,pi1 = my_EM(a= a1,b=b1,c=c1)

break after step: 16


In [30]:
print("a:")
print(af1)
print("b:")
print(bf1)
print("c:")
print(cf1)
print("pi:")
print(pi1)

a:
[0.02951906 0.00548094 0.965     ]
b:
[1.50747660e-01 8.49252340e-01 8.54105684e-12]
c:
[9.07466660e-01 9.25333401e-02 1.29028877e-15]
pi:
[0.27       0.15       0.05935875 0.08064125 0.09       0.05
 0.06       0.01       0.03017781 0.04964437 0.02017781 0.03
 0.00967937 0.01032063 0.02017781 0.04982219 0.01      ]


## Comment:
From the result above, we can see the EM algorithm manages to output satisfactory parameters for this toy problem. Even after changing the initial conditions of a,b,c, the algorthm seems to be able to get similar results.